In [10]:
# General imports
import os
import time
import datetime
from os.path import expanduser
import pandas
import pickle
import json
import numpy
from matplotlib import pyplot as plt

import mujoco_py
from gym.envs.robotics import rotations

import rospy
from sensor_msgs.msg import JointState
from std_msgs.msg import Float64MultiArray
from std_msgs.msg import String
from geometry_msgs.msg import WrenchStamped
from trajectory_msgs.msg import JointTrajectory, JointTrajectoryPoint
from realsense2_camera.msg import CustomVector
from realsense2_camera.msg import EstimatedPose

In [8]:
def normalize_rad(angles):
    angles = numpy.array(angles)
    angles = angles % (2*numpy.pi)
    angles = (angles + 2*numpy.pi) % (2*numpy.pi)
    for i in range(len(angles)):
        if (angles[i] > numpy.pi):
            angles[i] -= 2*numpy.pi
    return angles

In [12]:
HOME_PATH = os.getenv("HOME")
PROJECT_PATH = os.path.join(*[HOME_PATH, "DRL_AI4RoMoCo"])
SAVE_PATH = os.path.join(*[PROJECT_PATH, "code", "data","TEST_Reality"])

goal = numpy.array([0.69539974, -0.83325208,  1.17396257, -1.57092218,  0.02244061, 3.14048138])

XML_MODEL_PATH = os.path.join(*[HOME_PATH, "DRL_AI4RoMoCo", "code", "environment","UR10_single","ur10_heg.xml"])   #Force
#XML_MODEL_PATH = os.path.join(*[HOME_PATH, "DRL_AI4RoMoCo", "code", "environment","UR10_single_position","ur10_heg_position.xml"])

env_config = "reality_config_force.yml"
#checkpoint_number = 4950
#model = "PPO_NoisyPositionEnv_0_2020-09-18_20-14-55ej_cw9mo"
control_type = env_config.split("_")[2].split(".")[0]

ENV_CONFIG = os.path.join(*[HOME_PATH, "AGENT_TESTS", env_config])
#ENV_CONFIG = os.path.join(*[HOME_PATH, "AGENT_TESTS", "PPO_ForceEnv_0_2020-09-10_15-29-51_j1bhegb", config])

In [3]:
def get_dq(dx):
    jacp = sim.data.get_body_jacp(name="gripper_dummy_heg").reshape(3, 6)
    jacr = sim.data.get_body_jacr(name="gripper_dummy_heg").reshape(3, 6)
    jac = numpy.vstack((jacp, jacr))
    dq = numpy.linalg.lstsq(jac, dx)[0].reshape(6, )
    return dq

def go_smooth(q):
    rate = rospy.Rate(125)
    dq = q - actual_q
    while numpy.linalg.norm(dq)>0.005:
        dq = q - actual_q
        dq_ = numpy.clip(dq, -0.0001, 0.0001)
        control(ctrl_q+dq_)
        rate.sleep()
    control(q)
    
def set_state(qpos):
    #assert qpos.shape == (model.nq,)
    old_state = sim.get_state()
    new_state = mujoco_py.MjSimState(old_state.time, qpos, old_state.qvel,
                                     old_state.act, old_state.udd_state)
    sim.set_state(new_state)
    sim.forward()

In [4]:
model = mujoco_py.load_model_from_path(XML_MODEL_PATH)
sim = mujoco_py.MjSim(model)
q_init = numpy.array([0, -1.3, 2.1, -0.80, 1.5708, 0.0])

ctrl_q = q_init
set_state(q_init)

viewer = mujoco_py.MjViewer(sim)
viewer.render()

Creating window glfw


In [5]:
def q_callback(data):
    '''Callback for the joint angles, velocities and accelerations'''
    global actual_q
    actual_q = numpy.array(data.position)[[2,1,0,3,4,5]]
    set_state(actual_q)
    #viewer.render()

def pose_callback(data):
    '''ROS Callback function for the pose_estimation'''
    global pose
    pose = numpy.array([(-1)*data.tx, data.ty, data.tz]) #, data.rx, data.ry, data.rz])
        
def ft_callback(data):
    '''ROS Callback function for the force-torque data'''
    global ft_values
    ft_values = (-1)* numpy.array([data.wrench.force.x, data.wrench.force.y, data.wrench.force.z, data.wrench.torque.x, data.wrench.torque.y, data.wrench.torque.z])  

def get_real_obs():
    
    set_state(actual_q)
    
    real_pos = pose.copy()
    
    x_mat = sim.data.get_body_xmat("gripper_dummy_heg")
    rpy =  normalize_rad(rotations.mat2euler(x_mat))

    obs = numpy.concatenate([
                x_mat.dot(real_pos), x_mat.dot(normalize_rad(rpy-goal[3:])), ft_values.copy()
            ])
    return obs

In [6]:
rospy.init_node("DRL_AI4RoMoCo", anonymous=True)
rospy.Subscriber("/joint_states", JointState, q_callback)
rospy.Subscriber("/ft300_force_torque", WrenchStamped, ft_callback)
rospy.Subscriber("/pose_estimation", EstimatedPose, pose_callback)
#rospy.Subscriber("/observation", CustomVector, observation_callback)
#control_publisher = rospy.Publisher("/joint_group_position_controller/command", Float64MultiArray, queue_size=1)
#traj_control_publisher = rospy.Publisher("/pos_traj_controller/command", JointTrajectory, queue_size=1)
#ft_zero_publisher = rospy.Publisher("/ur_hardware_interface/script_command", String, queue_size=1)
rate = rospy.Rate(25)

In [13]:
get_real_obs()

array([-1.28194379e-01,  2.95363139e-03,  2.58527672e-03, -2.48634502e-04,
        1.27166100e-03, -2.23066894e-02, -2.99000000e+00,  7.80000000e+00,
       -1.93099990e+01,  1.10000000e-01,  1.77000000e-01, -9.00000000e-03])